# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [7]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


In [9]:
adult_dt.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [10]:
X = adult_dt.drop(columns=['income'])
Y = adult_dt['income']

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=42)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Y_train shape: {Y_train.shape}")
print(f"Y_test shape: {Y_test.shape}")

X_train shape: (22792, 14)
X_test shape: (9769, 14)
Y_train shape: (22792,)
Y_test shape: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

By setting the random_state, you make sure that every time you split the data, you get the same training and testing sets. This is crucial for experiments where you want consistent and comparable results.Besides, if you're testing different models or algorithms on the same data, using the same random_state allows you to compare their performance fairly since they are trained and evaluated on the same subsets.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [13]:
adult_dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  int32 
dtypes: int32(1), int64(6), object(8)
memory usage: 3.6+ MB


In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder

In [ ]:
# Define the transformers for numerical and categorical columns
num_transforms = Pipeline(steps=[
    ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

cat_transforms = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

# Define the ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('num_transforms', num_transforms, ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']),
    ('cat_transforms', cat_transforms, ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'])
])


In [33]:
# Check the created ColumnTransformer
print(preprocessor)

ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('knn_imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('scaler', RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
          

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [30]:
from sklearn.ensemble import RandomForestClassifier

# Create the final model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  # The ColumnTransformer from the previous step
    ('classifier', RandomForestClassifier())  # The classifier step
])

# Check the created pipeline
print(model_pipeline)


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('knn_imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  P

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [32]:
from sklearn.model_selection import cross_validate
import numpy as np

# Specify the scoring metrics
scoring_metrics = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy']

# Perform cross-validation with 5 folds
cv_results = cross_validate(
    model_pipeline, X_train, Y_train,
    cv=5, scoring=scoring_metrics,
    return_train_score=True
)

# Report the training and validation results
for metric in scoring_metrics:
    print(f"{metric}:")
    print(f"  Training: {np.mean(cv_results[f'train_{metric}']):.4f} ± {np.std(cv_results[f'train_{metric}']):.4f}")
    print(f"  Validation: {np.mean(cv_results[f'test_{metric}']):.4f} ± {np.std(cv_results[f'test_{metric}']):.4f}")
    print()


c:\Users\Hossein-Bg\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Hossein-Bg\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


neg_log_loss:
  Training: -0.0817 ± 0.0002
  Validation: -0.3698 ± 0.0219

roc_auc:
  Training: 1.0000 ± 0.0000
  Validation: 0.9036 ± 0.0022

accuracy:
  Training: 1.0000 ± 0.0000
  Validation: 0.8535 ± 0.0033

balanced_accuracy:
  Training: 1.0000 ± 0.0000
  Validation: 0.7757 ± 0.0038



Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [34]:
import pandas as pd

# Extract fold-level results and organize them into a DataFrame
fold_results = pd.DataFrame(cv_results)

# Sort the DataFrame by the negative log loss of the test set (ascending)
fold_results_sorted = fold_results.sort_values(by='test_neg_log_loss', ascending=True)

# Display the sorted DataFrame
print(fold_results_sorted[['train_neg_log_loss', 'test_neg_log_loss', 'train_roc_auc', 'test_roc_auc',
                           'train_accuracy', 'test_accuracy', 'train_balanced_accuracy', 'test_balanced_accuracy']])


   train_neg_log_loss  test_neg_log_loss  train_roc_auc  test_roc_auc  \
4           -0.082062          -0.401103            1.0      0.901449   
2           -0.081506          -0.381763            1.0      0.901781   
1           -0.081659          -0.367658            1.0      0.902183   
3           -0.081985          -0.363595            1.0      0.906795   
0           -0.081461          -0.334632            1.0      0.905621   

   train_accuracy  test_accuracy  train_balanced_accuracy  \
4             1.0       0.855200                      1.0   
2             1.0       0.854103                      1.0   
1             1.0       0.847774                      1.0   
3             1.0       0.857613                      1.0   
0             1.0       0.852819                      1.0   

   test_balanced_accuracy  
4                0.777356  
2                0.777248  
1                0.768607  
3                0.779872  
0                0.775625  


Calculate the mean of each metric. 

In [35]:
# Calculate the mean of each metric across folds
metric_means = fold_results.mean()

# Display the mean for each metric
print("Mean of each metric:")
print(metric_means[['train_neg_log_loss', 'test_neg_log_loss', 
                    'train_roc_auc', 'test_roc_auc', 
                    'train_accuracy', 'test_accuracy', 
                    'train_balanced_accuracy', 'test_balanced_accuracy']])


Mean of each metric:
train_neg_log_loss        -0.081734
test_neg_log_loss         -0.369750
train_roc_auc              1.000000
test_roc_auc               0.903566
train_accuracy             1.000000
test_accuracy              0.853502
train_balanced_accuracy    1.000000
test_balanced_accuracy     0.775742
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [37]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Fit the model pipeline on the training data
model_pipeline.fit(X_train, Y_train)

# Generate predictions and prediction probabilities on the test set
Y_pred = model_pipeline.predict(X_test)
Y_pred_proba = model_pipeline.predict_proba(X_test)

# Calculate the performance metrics
test_metrics = {
    'neg_log_loss': -log_loss(Y_test, Y_pred_proba),
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    'accuracy': accuracy_score(Y_test, Y_pred),
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred)
}

# Display results as a dictionary
print("Testing performance metrics:")
print(test_metrics)


Testing performance metrics:
{'neg_log_loss': -0.3842871640537188, 'roc_auc': 0.9003313166234516, 'accuracy': 0.8548469648889344, 'balanced_accuracy': 0.7748129514627378}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

 Recoding the target variable income from a categorical format ('>50K' and '<=50K') to a binary format (1 and 0) simplifies the subsequent analysis and model training processes. Many machine learning algorithms expect numerical targets, so converting the target to a binary format upfront reduces the need for this step later.

 Assigning the transformed income column immediately as the data is loaded means that the resulting DataFrame is ready for model training and evaluation without additional preprocessing. This saves time and keeps the data preparation steps more readable and organized.

 Using a lambda function within assign() keeps the recoding step compact and easy to understand. It clearly shows that the income variable is being recoded as a binary variable where '>50K' becomes 1 (high income) and '<=' becomes 0 (low income).

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.